# **Machine Learning Based Topic Recognition For Researchers**

##Importing Libraries

In [ ]:
# panda lib used for data computation
import pandas as pd
import matplotlib.pyplot as plt
# plot graph
%matplotlib inline
plt.style.use('seaborn-whitegrid')
import xgboost as xgb
import gc
from tqdm import tqdm
# progress bar
from sklearn.svm import SVC
# clustering algo to automatically assume parameter
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
# keras: open source lib to provide ai neural network
# long short term memmory used in predict future value based on previous(time stmped)
# sequence precting,less complex than lstm
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
# layer used in nlp lang model
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
from keras.utils import np_utils
# used to convert label data array in vector
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# tfidvectorizer: transform text to feature vector,convert to countable vector
from sklearn.decomposition import TruncatedSVD
# generarte matrix with specific num of col
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
# variation of k fold returns stratified fold
from sklearn.metrics import accuracy_score, f1_score,classification_report
from sklearn.model_selection import KFold, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB

from lightgbm import LGBMRegressor
# grading boosting frmwrk to reduce memory space and increase efficency
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings

warnings.simplefilter('ignore')
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
# breakdown of big value in small val
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
# stop training when  monitered matrix ha stopped improving
from nltk import word_tokenize
# nltk: natral lan tool kit used for nlp
# work_tokenize: split sentence to word
from nltk.corpus import stopwords
# top 40 words which are not relevant for prediction(e.g a,an the etc)
from nltk.stem.snowball import SnowballStemmer
# string processing lan for creating steeming algo(process of reducing inflrtion of word to there root form such as maping of group of word to same stem even if stem is not valid word)


In [ ]:
import nltk
nltk.download('stopwords')
# download stop words a,an the etc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = stopwords.words('english')
import seaborn as sns
# data vizulization lib based on matplot lib
import string
!pip install Unidecode
# better result compared to directly char
import re
# regular expression,specify strings that match it

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 5.5 MB/s 


In [ ]:
!pip install scikit-multilearn
# multi labeled classification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 3.8 MB/s 


In [ ]:
from skmultilearn.problem_transform import LabelPowerset # initialize label powerset multi-label classifier
%matplotlib inline

## Importing datasets

In [ ]:
import os
# to interact with underlined os

In [ ]:
from google.colab import files
uploaded = files.upload()
# upload file from user side for training and testing purpose(main dataset)

Saving Test.csv to Test.csv


In [ ]:
import io
# input output operartion
test = pd.read_csv(io.BytesIO(uploaded['Test.csv']))
# read test data

In [ ]:
from google.colab import files
# uploading training data
uploaded = files.upload()

Saving Train.csv to Train.csv


In [ ]:
import io
train = pd.read_csv(io.BytesIO(uploaded['Train.csv']))
# reading training data

In [ ]:
from google.colab import files
# uploading tags file e.g labels 
uploaded = files.upload()

Saving Tags.csv to Tags.csv


In [ ]:
import io
tag = pd.read_csv(io.BytesIO(uploaded['Tags.csv']))
# reading tags data

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sample_sub.csv to sample_sub.csv


In [ ]:
import io
ss = pd.read_csv(io.BytesIO(uploaded['sample_sub.csv']))

## Implementation

In [ ]:
#Display all the columns of the train dataset
# our result willbe among these topics only
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14004 entries, 0 to 14003
Data columns (total 31 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   id                                            14004 non-null  int64 
 1   ABSTRACT                                      14004 non-null  object
 2   Computer Science                              14004 non-null  int64 
 3   Mathematics                                   14004 non-null  int64 
 4   Physics                                       14004 non-null  int64 
 5   Statistics                                    14004 non-null  int64 
 6   Analysis of PDEs                              14004 non-null  int64 
 7   Applications                                  14004 non-null  int64 
 8   Artificial Intelligence                       14004 non-null  int64 
 9   Astrophysics of Galaxies                      14004 non-null  int64 
 10

In [ ]:
#We divided the columns into TARGET_COLS and TOPIC_COLS  
# do tabel hai ,maintabele (topic_cols hai aur uska subtopics taget_cols hai)
ID_COL = 'id'

TARGET_COLS = ['Analysis of PDEs', 'Applications',
               'Artificial Intelligence', 'Astrophysics of Galaxies',
               'Computation and Language', 'Computer Vision and Pattern Recognition',
               'Cosmology and Nongalactic Astrophysics',
               'Data Structures and Algorithms', 'Differential Geometry',
               'Earth and Planetary Astrophysics', 'Fluid Dynamics',
               'Information Theory', 'Instrumentation and Methods for Astrophysics',
               'Machine Learning', 'Materials Science', 'Methodology', 'Number Theory',
               'Optimization and Control', 'Representation Theory', 'Robotics',
               'Social and Information Networks', 'Statistics Theory',
               'Strongly Correlated Electrons', 'Superconductivity',
               'Systems and Control']
#The research article abstracts are sourced from the following 4 topics
TOPIC_COLS = ['Computer Science', 'Mathematics', 'Physics', 'Statistics']

In [ ]:
#Sorting in descending order according to values corresponding to topics 
train[TOPIC_COLS].sum().sort_values(ascending=False)
# hm jo pargraph denge usme yeh har word pe jaiega aur group krega ki woh word kis topic se related hai lis cs,math,physics etc, jo aur number count hoke woh decending order me arrange ho jaega like agar work ml bahut baar aa rha tog cs topuc aa jaega

Computer Science    5884
Physics             3856
Statistics          3794
Mathematics         2831
dtype: int64

In [ ]:
100 * (train[TARGET_COLS].sum()/(train.shape[0])).sort_values(ascending=False)
# ab manno hme pta chl gya ki computer science topic se related topic hai phir abb woh cs ke andar ke sub topics ko serach krega aur most closed topic output laake dega

Machine Learning                                27.313625
Artificial Intelligence                          9.825764
Robotics                                         6.812339
Computer Vision and Pattern Recognition          6.705227
Strongly Correlated Electrons                    6.376750
Materials Science                                5.534133
Computation and Language                         4.605827
Cosmology and Nongalactic Astrophysics           4.520137
Optimization and Control                         4.434447
Social and Information Networks                  4.420166
Analysis of PDEs                                 4.334476
Applications                                     4.227364
Astrophysics of Galaxies                         4.098829
Methodology                                      4.098829
Systems and Control                              4.041702
Differential Geometry                            3.963153
Superconductivity                                3.870323
Statistics The

## **1] Using Count Vectorizer** 
It is used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text. It creates a matrix in which each unique word is represented by a column of the matrix, and each text sample from the document is a row in the matrix. The value of each cell is nothing but the count of the word in that particular text sample. 

In [ ]:
vec = CountVectorizer(max_features=10000)
combined = list(train['ABSTRACT']) + list(test['ABSTRACT'])
vec.fit(combined)

CountVectorizer(max_features=10000)

In [ ]:
trn, val = train_test_split(train, test_size=0.2, random_state=2)

In [ ]:
trn_abs = vec.transform(trn['ABSTRACT'])
val_abs = vec.transform(val['ABSTRACT'])
tst_abs = vec.transform(test['ABSTRACT'])

trn_abs

<11203x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 1033404 stored elements in Compressed Sparse Row format>

In [ ]:
vec.get_feature_names()

['00',
 '000',
 '0001',
 '001',
 '005',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '100',
 '1000',
 '101',
 '102',
 '10x',
 '11',
 '110',
 '111',
 '112',
 '115',
 '118',
 '12',
 '120',
 '128',
 '13',
 '130',
 '14',
 '140',
 '15',
 '150',
 '1500',
 '16',
 '160',
 '17',
 '170',
 '18',
 '180',
 '19',
 '1977',
 '1986',
 '1991',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '1d',
 '1t',
 '20',
 '200',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '21',
 '22',
 '23',
 '230',
 '24',
 '240',
 '25',
 '250',
 '2500',
 '256',
 '26',
 '2600',
 '27',
 '28',
 '29',
 '2_',
 '2d',
 '2g',
 '2h',
 '2k',
 '2m',
 '2mass',
 '2n',
 '2p',
 '2s',
 '2x',
 '30',
 '300',
 '3000',
 '31',
 '32',
 '33',
 '34',
 '35',
 '350',
 '36',
 '360',
 '37',
 '38',
 '39',
 '3b',
 '3d',
 '3x',
 '40',
 '400',
 '4000',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '

In [ ]:
clf = OneVsRestClassifier(LogisticRegression(C = 10, n_jobs=-1))
clf.fit(trn_abs, trn[TARGET_COLS])

OneVsRestClassifier(estimator=LogisticRegression(C=10, n_jobs=-1))

In [ ]:
val_preds = clf.predict(val_abs)
f1_score(val[TARGET_COLS], val_preds, average='micro')

0.623196768609348

**2]Using Tfidf Vectorizer** - Term frequency-inverse document frequency is a text vectorizer that transforms the text into a usable vector.


*    Term frequency represents every text from the data as a matrix whose rows are the number of documents and columns are the number of distinct terms throughout all documents.
*   Inverse document frequency (IDF) is the weight of a term, it aims to reduce the weight of a term if the term’s occurrences are scattered throughout all the documents.



In [ ]:
vec = TfidfVectorizer(max_features=10000)
_ = vec.fit(list(train['ABSTRACT']) + list(test['ABSTRACT']))

trn_abs = vec.transform(trn['ABSTRACT'])
val_abs = vec.transform(val['ABSTRACT'])
tst_abs = vec.transform(test['ABSTRACT'])

clf = OneVsRestClassifier(LogisticRegression(C = 10, n_jobs=-1))
_ = clf.fit(trn_abs, trn[TARGET_COLS])

val_preds = clf.predict(val_abs)
f1_score(val[TARGET_COLS], val_preds, average='micro')

0.6300845429893125

## **Sequential Neural Network**

### Building Corpus

In [ ]:
train['text'] = ' '
test['text'] = ' '

#this is our corpus basically
train['text'] += train['ABSTRACT'] 
test['text'] += test['ABSTRACT']

#Splitting the training data into test and train dataset where 20% is used for testing and 80% for training
trn, val = train_test_split(train, test_size=0.2, random_state=2)

trn.columns

Index(['id', 'ABSTRACT', 'Computer Science', 'Mathematics', 'Physics',
       'Statistics', 'Analysis of PDEs', 'Applications',
       'Artificial Intelligence', 'Astrophysics of Galaxies',
       'Computation and Language', 'Computer Vision and Pattern Recognition',
       'Cosmology and Nongalactic Astrophysics',
       'Data Structures and Algorithms', 'Differential Geometry',
       'Earth and Planetary Astrophysics', 'Fluid Dynamics',
       'Information Theory', 'Instrumentation and Methods for Astrophysics',
       'Machine Learning', 'Materials Science', 'Methodology', 'Number Theory',
       'Optimization and Control', 'Representation Theory', 'Robotics',
       'Social and Information Networks', 'Statistics Theory',
       'Strongly Correlated Electrons', 'Superconductivity',
       'Systems and Control', 'text'],
      dtype='object')

### Tokenizing the text

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#100000 is the max. no. of words to keep in the tokenized list
tok = Tokenizer(num_words = 1000000) 
tok.fit_on_texts(train['text'].str.lower().tolist() + test['text'].str.lower().tolist()) #updates the internal dictionary based on a list of words

vocab_size = len(tok.word_index) + 1
vocab_size

51665

### Converting into column vectors

In [ ]:
# For each word in each sentences, converting into column vector and producing the one hot encoding
X_trn = tok.texts_to_sequences(trn['text'])
X_val = tok.texts_to_sequences(val['text'])
X_test = tok.texts_to_sequences(test['text'])

In [ ]:
maxlen = 200 #maximum length of all sequences(i.e, to what length is each sentence padded upto)
X_trn = pad_sequences(X_trn, maxlen=maxlen)
X_val = pad_sequences(X_val, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

X_test

array([[    0,     0,     0, ...,   280,   965,    53],
       [    0,     0,     0, ...,  1278,   423,  4957],
       [    1, 10832,    75, ...,     5,  9884,  4154],
       ...,
       [   36,  1514,    10, ...,    99,   264,  2804],
       [    2,  7933,    22, ...,    62,   123,   125],
       [    0,     0,     0, ...,   412,  6056,   164]], dtype=int32)

### Creating a **Word Embedding Layer**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, SpatialDropout1D, LSTM
from keras.metrics import categorical_accuracy

embedding_dim = 50 # taken 50 'features'
vocab_size = len(tok.word_index) + 1

# Allow algorithms to distinguish context and figure out what words are related to each other
model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_dim,
                    input_length=maxlen))

# Using back propagation method, our model will start learning
model.add(Flatten())
model.add(Dense(200, activation='relu', name = 'Fully_Connected'))
model.add(Dense(25, activation='sigmoid', name = 'Output'))
model.compile(optimizer=tf.keras.optimizers.Adam(lr = 1e-3),
              loss='binary_crossentropy',
              metrics=['categorical_accuracy'],
              )

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 50)           2583250   
                                                                 
 flatten (Flatten)           (None, 10000)             0         
                                                                 
 Fully_Connected (Dense)     (None, 200)               2000200   
                                                                 
 Output (Dense)              (None, 25)                5025      
                                                                 
Total params: 4,588,475
Trainable params: 4,588,475
Non-trainable params: 0
_________________________________________________________________


### Fitting Model

In [ ]:
model.fit(X_trn, trn[TARGET_COLS], validation_data=(X_val, val[TARGET_COLS]), verbose=True, epochs=20, batch_size=256,
          callbacks = [tf.keras.callbacks.ReduceLROnPlateau()])

Epoch 1/20
44/44 [==============================] - 6s 115ms/step - loss: 0.2790 - categorical_accuracy: 0.1379 - val_loss: 0.1920 - val_categorical_accuracy: 0.1678 - lr: 0.0010
Epoch 2/20
44/44 [==============================] - 5s 109ms/step - loss: 0.1783 - categorical_accuracy: 0.2027 - val_loss: 0.1681 - val_categorical_accuracy: 0.2563 - lr: 0.0010
Epoch 3/20
44/44 [==============================] - 5s 109ms/step - loss: 0.1461 - categorical_accuracy: 0.3226 - val_loss: 0.1489 - val_categorical_accuracy: 0.3027 - lr: 0.0010
Epoch 4/20
44/44 [==============================] - 5s 109ms/step - loss: 0.1183 - categorical_accuracy: 0.4711 - val_loss: 0.1341 - val_categorical_accuracy: 0.3563 - lr: 0.0010
Epoch 5/20
44/44 [==============================] - 5s 110ms/step - loss: 0.0867 - categorical_accuracy: 0.6737 - val_loss: 0.1223 - val_categorical_accuracy: 0.4288 - lr: 0.0010
Epoch 6/20
44/44 [==============================] - 5s 109ms/step - loss: 0.0571 - categorical_accuracy: 

### F1 Score , Precision and Recall


*   F1 Score = 2*((precision*recall)/(precision+recall))
*   Recall = True positive yes which means when it is yes, how many times it is actually yes
*   Precision = If its yes, how often it's correct 





In [ ]:
import numpy as np
def get_best_thresholds(true, preds):
  thresholds = [i/100 for i in range(100)]
  best_thresholds = []
  for idx in range(25):
    f1_scores = [f1_score(true[:, idx], (preds[:, idx] > thresh) * 1) for thresh in thresholds]
    best_thresh = thresholds[np.argmax(f1_scores)]
    best_thresholds.append(best_thresh)
  return best_thresholds

val_preds = model.predict(X_val)
best_thresholds = get_best_thresholds(val[TARGET_COLS].values, val_preds)
for i, thresh in enumerate(best_thresholds):
  val_preds[:, i] = (val_preds[:, i] > thresh) * 1
f1_score(val[TARGET_COLS], val_preds, average='micro')

0.5768793430195831

### Saving the model

In [ ]:
model.save("NLP_word_embeddings.h5")

##End of Model

## Deployment in **FLASK**

In [ ]:
from tensorflow.keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np

In [ ]:
maxlen = 200
#100000 is the max. no. of words to keep in the tokenized list
tok = Tokenizer(num_words = 1000000) 
tok.fit_on_texts(train['ABSTRACT'].str.lower().tolist() + test['ABSTRACT'].str.lower().tolist())

word_embedding_model = load_model("NLP_word_embeddings.h5")

demo_data1 = [""" hello world """]


demo_input = tok.texts_to_sequences(demo_data1)

demo_input =  pad_sequences(demo_input, maxlen=maxlen)

pred = word_embedding_model.predict(demo_input)

In [ ]:
print(TARGET_COLS[np.argmax(pred)]) #highest probabilty prediction
print(pred[0,np.argmax(pred)])

Number Theory
0.059233814


In [ ]:
def test(text):
  demo_input = tok.texts_to_sequences([text])
  demo_input =  pad_sequences(demo_input, maxlen=maxlen)
  pred = word_embedding_model.predict(demo_input)
  # print(TARGET_COLS[np.argmax(pred)], pred[0,np.argmax(pred)])
  data = {}
  data['topic'] = TARGET_COLS[np.argmax(pred)]
  data['confidence'] = str(pred[0,np.argmax(pred)])
  return data

In [ ]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install -U flask-cors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, redirect, url_for, request
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

run_with_ngrok(app)
@app.route('/', methods=['POST'])
def login():
  text = request.form['text']
  data = test(text)
  print(data);
  return data

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://cd5d-34-75-177-55.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [25/Aug/2022 13:47:25] "POST / HTTP/1.1" 200 -


{'topic': 'Machine Learning', 'confidence': '0.8903581'}


...

{'topic': 'Computation and Language', 'confidence': '0.5558007'}


INFO:werkzeug:127.0.0.1 - - [25/Aug/2022 14:01:43] "POST / HTTP/1.1" 200 -


{'topic': 'Cosmology and Nongalactic Astrophysics', 'confidence': '0.01485461'}


INFO:werkzeug:127.0.0.1 - - [25/Aug/2022 14:02:24] "POST / HTTP/1.1" 200 -


{'topic': 'Systems and Control', 'confidence': '0.080346376'}
